# **線性迴歸模型 (Model Representation)**

<br>

## **目標 (Goals)**
在這個實驗中，你將會:
- 學習如何實作單變數的線性迴歸模型 $f_{w,b}$

<br>

## **符號 (Notation)**
以下是你將會遇到的一些符號摘要。 

<div align="center">

| Notation  <img width=70/> | Description<img width=350/> | Python (if applicable) |
|:------------|:------------------------------------------------------------|:------|
|$a$| scalar, non bold ||
|$\mathbf{a}$ | vector, bold||
|**Regression**|||
|  $\mathbf{x}$ | Training Example feature values (in this lab - Size (1000 sqft))  | `x_train` |   
|  $\mathbf{y}$  | Training Example  targets (in this lab Price (1000s of dollars))  | `y_train` 
|  $x^{(i)}$, $y^{(i)}$ | $i_{th}$ Training Example | `x_i`, `y_i`|
| m | Number of training examples | `m`|
|  $w$  |  parameter : weight                                 | `w`    |
|  $b$           |  parameter: bias                                           | `b`    |     
| $f_{w,b}(x^{(i)})$ | The result of the model evaluation at $x^{(i)}$ parameterized by $w,b$: $f_{w,b}(x^{(i)}) = wx^{(i)}+b$  | `f_wb` | 

</div>

<br>

## **模組工具 (Tools)**
在這個實驗中，你將會使用：
- NumPy (廣受歡迎的科學運算函式庫)
- Matplotlib (常用的資料繪圖函式庫)

In [ ]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

#region 匯入相關模組
def find_repo_root(marker="README.md"):
    cur = Path.cwd()
    while cur != cur.parent:  # 防止無限迴圈，到達檔案系統根目錄就停
        if (cur / marker).exists():
            return cur
        cur = cur.parent
    return None


def import_data_from_github():
    import urllib.request, shutil
    
    def isRunningInColab() -> bool:
        return "google.colab" in sys.modules

    def isRunningInJupyterLab() -> bool:
        try:
            import jupyterlab
            return True
        except ImportError:
            return False
        
    def detect_env():
        from IPython import get_ipython
        if isRunningInColab():
            return "Colab"
        elif isRunningInJupyterLab():
            return "JupyterLab"
        elif "notebook" in str(type(get_ipython())).lower():
            return "Jupyter Notebook"
        else:
            return "Unknown"
        
    def get_utils_dir(env): 
        if env == "Colab": 
            if "/content" not in sys.path:
                sys.path.insert(0, "/content")
            return "/content/utils"
        else:
            return Path.cwd() / "utils"

    env = detect_env()
    UTILS_DIR = get_utils_dir(env)
    REPO_DIR = "Machine-Learning-Lab"

    shutil.rmtree(UTILS_DIR, ignore_errors=True)
    os.makedirs(UTILS_DIR, exist_ok=True)

    BASE = f"https://raw.githubusercontent.com/mz038197/{REPO_DIR}/main"
    urllib.request.urlretrieve(f"{BASE}/utils/deeplearning.mplstyle", f"{UTILS_DIR}/deeplearning.mplstyle")


repo_root = find_repo_root()

if repo_root is None:
    import_data_from_github()
    repo_root = Path.cwd()

os.chdir(repo_root)
print(f"✅ 切換工作目錄至 {Path.cwd()}")
sys.path.append(str(repo_root)) if str(repo_root) not in sys.path else None
print(f"✅ 加入到系統路徑")

plt.style.use('utils/deeplearning.mplstyle')
print("✅ 匯入模組及設定繪圖樣式")
#endregion 匯入相關模組    

✅ 切換工作目錄至 d:\Work\Python\Machine-Learning-Lab
✅ 加入到系統路徑
✅ 匯入模組及設定繪圖樣式


<br>

# **問題說明 (Problem statement)**

如同課堂講義所示，你將使用「房價預測」作為接下來學習範例。
本次實驗將使用一個非常簡單的資料集，僅包含兩筆資料：

* 一棟 1000 坪的房屋售價為 $300,0000

* 一棟 2000 坪的房屋售價為 $500,0000

這兩個資料點將構成我們的*資料集或訓練集*。

在本實驗中，房屋面積的單位以 1000 坪 表示，而價格的單位以 萬（$10000） 表示。


<div align="center">

| 大小 (1000 坪)     | 價格 (萬) |
| -------------------| ------------------------ |
| 1.0               | 300                      |
| 2.0               | 500                      |

</div>

你希望透過這兩個資料點，擬合出一條線性迴歸模型，<br>
這樣就能用它來預測其他房屋的價格——例如，一棟面積為 1200 坪 的房子。

請完成以下程式碼區塊，以建立你的 `x_train` 和 `y_train` 變數。

這些資料將以一維的 NumPy 陣列（NumPy arrays）形式儲存：

In [ ]:
# x_train is the input variable (size in 1000 ping)
# y_train is the target (price in 10000s of dollars)
x_train = np.array([1.0, 2.0])
y_train = np.array([300.0, 500.0])

In [ ]:
print(f"x_train = {x_train}")
print(f"y_train = {y_train}")

>**備註**: 本課程在輸出時，會經常使用 Python 的 f-string 格式化方式。

<br>

### **訓練樣本數量 (Number of training examples)** `m`
你將使用變數 `m` 來表示訓練樣本的數量。NumPy 陣列具有一個屬性 `.shape`，可用來查看陣列的維度 :

* `x_train.shape` 會回傳一個 Python 元組（tuple），其中包含每個維度的大小。

* `x_train.shape[0]` 則代表陣列的長度，也就是訓練樣本的數量。

如下範例所示：

In [ ]:
# m is the number of training examples
print(f"x_train.shape: {x_train.shape}")
m = x_train.shape[0]
print(f"訓練樣本的數量為: {m}")

你也可以使用 Python 的 `len()` 函式來取得樣本數量，如下所示：

In [ ]:
# m is the number of training examples
m = len(x_train)
print(f"訓練樣本的數量為: {m}")

<br>

### **訓練樣本 (Training example)** `x_i, y_i`

你將使用 $(x^{(i)}$, $y^{(i)})$ 來表示第 $i$ 個訓練樣本。

由於 Python 的索引是從 0 開始，因此 : 

- ( $x^{(0)}$, $y^{(0)}$ ) = $(1.0, 300.0)$

- ( $x^{(1)}$, $y^{(1)}$ ) = $(2.0, 500.0)$

要在 NumPy 陣列中存取某個值，可以透過索引指定位置。例如，若要取得 `x_train` 的第 0 個元素，可使用：`x_train[0]`。

接下來，請執行以下程式碼以取得第 $i$ 個訓練樣本：

In [ ]:
i = 0 # Change this to 1 to see (x^1, y^1)

x_i = x_train[i]
y_i = y_train[i]
print(f"(x^({i}), y^({i})) = ({x_i}, {y_i})")

<br>

### **繪製資料 (Plotting the data)**

你可以使用 `matplotlib` 函式庫中的 `scatter()` 函式來繪製這兩個資料點，如下方範例所示。

- 參數 `marker` 與 `c` 用來設定點的樣式，其中 `marker='x'` 和 `c='r'` 會將資料點顯示為紅色叉叉（預設是藍色圓點）。

你也可以使用 `matplotlib` 中的其他函式來設定圖表的標題與座標軸標籤

In [ ]:
# Plot the data points
plt.scatter(x_train, y_train, marker='x', c='r')
# Set the title
plt.title("Housing Prices")
# Set the y-axis label
plt.ylabel('Price (in 10000s of dollars)')
# Set the x-axis label
plt.xlabel('Size (1000 ping)')
plt.show()

<br>

## **模型函數 (Model function)**
如同課堂中所說，線性迴歸的模型函數（將輸入 `x` 對應到輸出 `y`）可表示為：
$$ f_{w,b}(x^{(i)}) = wx^{(i)} + b \tag{1} $$

上面的公式代表一條直線，不同的 $w$ (斜率)與 $b$ (截距) 值，都會對應到圖上不同的直線。<br/>

讓我們透過以下程式碼對這個概念建立更直覺的理解。首先設定 $w = 100$ 且 $b = 100$

>**Note: 你可以隨時回到這個程式區塊修改 w 和 𝑏 的值，觀察模型直線的變化**

In [ ]:
w = 100
b = 100
print(f"w: {w}")
print(f"b: {b}")

現在，讓我們計算兩個資料點的 $f_{w,b}(x^{(i)})$ 值。
你可以明確地寫出每個樣本的計算式：

- 對於 $x^{(0)}$ : `f_wb = w * x[0] + b`

- 對於 $x^{(1)}$ : `f_wb = w * x[1] + b`

但如果資料點很多，這樣的寫法會讓你寫到懷疑人生...<br>
讓我們改用 `for` 迴圈，試著定義並完成 `compute_model_output(x, w, b)` 的函式。<br>


> **Note**: `(ndarray (m,))` 代表形狀為 (m,) 的 NumPy 一維陣列。 `(scalar)` 代表純量（沒有維度的數值）。

> **Note**: `np.zero(n)` 會建立一個長度為 $n$ ，內容全為 0 的一維 NumPy 陣列。

In [ ]:
def compute_model_output(x, w, b):
    """
    Computes the prediction of a linear model
    Args:
      x (ndarray (m,)): Data, m examples 
      w,b (scalar)    : model parameters  
    Returns
      f_wb (ndarray (m,)): model prediction
    """
    m = x.shape[0]
    f_wb = np.zeros(m)
    for i in range(m):
        f_wb[i] = w * x[i] + b
        
    return f_wb

讓我們呼叫剛剛定義的 `compute_model_output` 函式，並將結果繪製出來：

In [ ]:
tmp_f_wb = compute_model_output(x_train, w, b,)

# Plot our model prediction
plt.plot(x_train, tmp_f_wb, c='b',label='Our Prediction')

# Plot the data points
plt.scatter(x_train, y_train, marker='x', c='r',label='Actual Values')

# Set the title
plt.title("Housing Prices")
# Set the y-axis label
plt.ylabel('Price (in 10000s of dollars)')
# Set the x-axis label
plt.xlabel('Size (1000 ping)')
plt.legend()
plt.show()

如你所見， $w = 100$ 且 $b = 100$ 的設定，並不會得到一條能貼合我們資料的直線。

<br>

### **挑戰 (Challenge)**
試著嘗試不同的 $w$ 和 $b$ 值。這些值應該要是多少，才能使直線貼合我們的資料?

<details>
<summary>
    <font size='3', color='darkgreen'><b>Hints</b></font>
</summary>
    <p>
    <ul>
        <li>Try w = 200 and b = 100 </li>
    </ul>
    </p>

<br>

### **預測 (Prediction)**
現在已經有了模型，我們就可以用它來進行預測! 讓我們來預測一棟 1200 坪 的房子價格。

由於資料中的面積單位是 以千坪（1000 ping）為單位，
因此輸入值 $x$ = 1.2

In [ ]:
w = 200                         
b = 100    
x_i = 1.2
cost_1200_ping = w * x_i + b    

print(f"${cost_1200_ping:.0f} 萬")

<br>

# **恭喜！**
在這個實驗中，你學到了以下內容：

 - 線性迴歸（Linear Regression）是一種建立 **特徵（features）與目標（targets）** 之間關係的模型

     - 在本範例中，特徵是「房屋面積」，目標是「房屋價格」。

     - 對於單變數線性迴歸，模型包含兩個參數：$w$ 和 $b$，它們的值是利用 **訓練資料（training data）** 來擬合出來的。
     
     - 一旦模型參數 $w$ 和 $b$ 確定了，便可用該模型來預測新的資料 (novel data)。

<br>